In [2]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical

Using TensorFlow backend.


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
model_input=[0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
model_output=[3]
train_data='seoul_housing_integrated.csv'

In [7]:
train_dataX = pd.read_csv(train_data, usecols = model_input)
train_dataY = pd.read_csv('temp.csv')

In [108]:
train_dataX.head(5)


,법정동코드,전용면적(㎡),계약년월,층,건축년도,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
0,1117010100,83.04,201405,4,2012,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
1,1117010100,40.99,201411,1,1973,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
2,1117010100,68.61,201407,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
3,1117010100,76.27,201501,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
4,1117010100,76.27,201503,1,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0


In [8]:
code_list = train_dataX['법정동코드']
code_list = code_list.drop_duplicates()
code = {}
cnt = 0 
for co in code_list:
    code[co]=cnt
    cnt = cnt + 1
    
X = train_dataX

In [9]:
X['나이'] = X['계약년월']/100 - X['건축년도']
X['나이'] = X['나이'].astype(int)

In [10]:
X = X.drop(columns=['계약년월', '건축년도'], axis=1)

In [11]:
idx = 0
for c in X['법정동코드']:
    try :
        #print(c)
        X.at[idx, '법정동코드'] = code[c]
        #print(code[c])
        idx = idx + 1
    except :
        continue

In [85]:
X = train_dataX

,법정동코드,전용면적(㎡),계약년월,층,건축년도,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
0,0,83.04,201405,4,2012,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
1,0,40.99,201411,1,1973,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
2,0,68.61,201407,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
3,0,76.27,201501,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
4,0,76.27,201503,1,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
5,0,163.27,201410,3,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
6,0,168.41,201412,2,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
7,0,166.56,201503,2,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
8,0,166.56,201503,3,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
9,0,83.69,201408,4,2005,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0


In [12]:
X.head(3)

,법정동코드,전용면적(㎡),층,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원,나이
0,0,83.04,4,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,2
1,0,40.99,1,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,41
2,0,68.61,5,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,8


In [80]:
train_dataY['price'].head(5)

0    45700
1    16000
2    42000
3    49800
4    50000
Name: price, dtype: int64

In [13]:
Y = train_dataY['price']
Y = Y/1000
Y = Y.astype(int)

In [76]:
XX = X.drop(columns=['법정동코드', '전용면적(㎡)', '층',  '나이'])

In [77]:
X_train, X_test , y_train, y_test = train_test_split(XX, Y, test_size = 0.2, random_state=123456)

In [78]:
np.random.seed(123456)

model = Sequential()

model.add(Dense(40, input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

model.compile(loss='mean_absolute_error', optimizer='adam')

In [79]:
model.fit(X_train, y_train, batch_size=2048, epochs=200, verbose=1)

Epoch 1/200
219070/219070 [==============================] - 1s 3us/step - loss: 51.8484
Epoch 2/200
219070/219070 [==============================] - 0s 1us/step - loss: 28.9920
Epoch 3/200
219070/219070 [==============================] - 0s 1us/step - loss: 26.9712
Epoch 4/200
219070/219070 [==============================] - 0s 1us/step - loss: 25.3761
Epoch 5/200
219070/219070 [==============================] - 0s 1us/step - loss: 24.6223
Epoch 6/200
219070/219070 [==============================] - 0s 1us/step - loss: 24.3490
Epoch 7/200
219070/219070 [==============================] - 0s 1us/step - loss: 24.1662
Epoch 8/200
219070/219070 [==============================] - 0s 1us/step - loss: 23.9879
Epoch 9/200
219070/219070 [==============================] - 0s 1us/step - loss: 23.8019
Epoch 10/200
219070/219070 [==============================] - 0s 1us/step - loss: 23.6019
Epoch 11/200
219070/219070 [==============================] - 0s 1us/step - loss: 23.4128
Epoch 12/200
219070

219070/219070 [==============================] - 0s 1us/step - loss: 19.8977
Epoch 93/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8929
Epoch 94/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8807
Epoch 95/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8780
Epoch 96/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.9005
Epoch 97/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8696
Epoch 98/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8662
Epoch 99/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8628
Epoch 100/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8556
Epoch 101/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8467
Epoch 102/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.8402
Epoch 103/200
219070

219070/219070 [==============================] - 0s 1us/step - loss: 19.6502
Epoch 183/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6409
Epoch 184/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6454
Epoch 185/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6472
Epoch 186/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6466
Epoch 187/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6348
Epoch 188/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6463
Epoch 189/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6370
Epoch 190/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6305
Epoch 191/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6305
Epoch 192/200
219070/219070 [==============================] - 0s 1us/step - loss: 19.6321
Epoch 193/200

In [80]:
dataY_predict = model.predict(X_test)

In [63]:
X_test

,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
121377,0.00,0.000000,0.000000,1.000000,0.000000,1.000000,10.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,0.00
89345,0.00,0.200000,2.000000,1.000000,0.000000,2.600000,7.600000,0.400000,0.000000,0.800000,0.000000,2.600000,0.000000,0.20
225145,2.50,0.500000,0.500000,0.500000,0.500000,2.000000,10.000000,1.000000,2.000000,1.500000,0.000000,1.500000,0.000000,0.00
28080,0.00,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.00
230205,0.00,0.000000,0.000000,0.000000,0.000000,1.500000,8.000000,0.000000,0.500000,0.500000,0.000000,3.000000,0.000000,0.00
186282,0.75,0.500000,0.000000,0.000000,0.000000,2.500000,8.000000,0.750000,0.250000,1.250000,0.250000,3.250000,0.000000,0.00
1765,0.00,0.000000,0.000000,0.000000,0.000000,3.000000,6.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.00
129214,0.00,0.000000,0.000000,0.250000,0.250000,1.750000,7.000000,0.000000,0.000000,1.000000,0.000000,2.500000,0.000000,0.25
156370,0.00,0.111111,0.000000,0.666667,0.111111,0.888889,8.888889,0.333333,0.000000,0.777778,0.333333,3.000000,0.000000,0.00
262431,0.00,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000,0.000000,0.000000,0.500000,0.000000,4.500000,0.000000,0.00


In [81]:
import sklearn.metrics as sm
print(sm.mean_absolute_error(dataY_predict,y_test))
print(sm.mean_squared_error(dataY_predict,y_test))

19.63425936520953
1176.5619811414037


In [83]:
X_test.drop_duplicates()

,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
121377,0.00,0.000000,0.000000,1.000000,0.000000,1.000000,10.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,0.00
89345,0.00,0.200000,2.000000,1.000000,0.000000,2.600000,7.600000,0.400000,0.000000,0.800000,0.000000,2.600000,0.000000,0.20
225145,2.50,0.500000,0.500000,0.500000,0.500000,2.000000,10.000000,1.000000,2.000000,1.500000,0.000000,1.500000,0.000000,0.00
28080,0.00,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.00
230205,0.00,0.000000,0.000000,0.000000,0.000000,1.500000,8.000000,0.000000,0.500000,0.500000,0.000000,3.000000,0.000000,0.00
186282,0.75,0.500000,0.000000,0.000000,0.000000,2.500000,8.000000,0.750000,0.250000,1.250000,0.250000,3.250000,0.000000,0.00
1765,0.00,0.000000,0.000000,0.000000,0.000000,3.000000,6.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.00
129214,0.00,0.000000,0.000000,0.250000,0.250000,1.750000,7.000000,0.000000,0.000000,1.000000,0.000000,2.500000,0.000000,0.25
156370,0.00,0.111111,0.000000,0.666667,0.111111,0.888889,8.888889,0.333333,0.000000,0.777778,0.333333,3.000000,0.000000,0.00
262431,0.00,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000,0.000000,0.000000,0.500000,0.000000,4.500000,0.000000,0.00


In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [18]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
movie_rf = RandomForestClassifier(n_estimators=50, criterion='entropy', oob_score = True, random_state=123456)
movie_rf.fit(X_train, y_train)

predicted = movie_rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'Out-of-bag score estimate: {movie_rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

print("특성:", X_train.columns)
print("특성 중요도:\n{}".format(movie_rf.feature_importances_))


Out-of-bag score estimate: 0.134
Mean accuracy score: 0.135
특성: Index(['법정동코드', '전용면적(㎡)', '층', '랜드마크', '영화관', '쇼핑센터', '대형마트', '백화점', '지하철',
       '차선수', '순환도로', '한강', '공원', '산', '학교', '교육특구', '대학병원', '나이'],
      dtype='object')
특성 중요도:
[0.03655477 0.30154894 0.26295046 0.00586641 0.00516469 0.01017849
 0.0082677  0.00381581 0.01725359 0.01597922 0.00759492 0.01713376
 0.01262597 0.00595142 0.01345019 0.00812562 0.00439442 0.26314363]


In [19]:
import xgboost as xgb


In [21]:
seoul_xgb = xgb.XGBClassifier(max_depth=3,
                              learning_rate=0.02,
                              n_estimators=30,
                              gamma=0.1,
                              booster='gbtree')
seoul_xgb.fit(X_train, y_train)
#    y_pred = movie_xgb.predict(y_train)
print("훈련 세트 정확도: {:.3f}".format(seoul_xgb.score(X_train, y_train)))
# y_pred = movie_xgb.predict(y_test)
print("테스트 세트 정확도: {:.3f}".format(seoul_xgb.score(X_test, y_test)))

KeyboardInterrupt: 